In [1]:
import pandas as pd
from pricer.pricer import Pricer
from pricer.pricer_factory import PricerFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing pricer_class
pricer_class = PricerFactory.build(Pricer.WEEKLY_CRYPTO_ROLLING)

In [3]:
backtest_start_date = datetime(2021,1,1)
backtest_end_date = datetime(2023,1,1)
current_year = datetime.now().year

In [4]:
## creating database objects
market = Market()
sec = SEC()
modeler_strat = UniversalModeler()

In [5]:
tickers = ["BTC"]

In [6]:
market.connect()
training_sets = []
for ticker in tqdm(tickers):
    try:
        prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = pricer_class.training_set(ticker,prices)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.02it/s]


In [7]:
data = pd.concat(training_sets)

In [8]:
training_data = data.dropna().copy().sort_values(["year","week"])

In [9]:
training_data = training_data.groupby(["year","quarter","week","ticker"]).mean().reset_index()

In [10]:
training_data

,year,quarter,week,ticker,price_prediction
0,2014,4,1,BTC,361.490206
1,2014,4,52,BTC,364.761057
2,2015,1,1,BTC,357.620447
3,2015,1,2,BTC,351.623932
4,2015,1,3,BTC,344.224964
...,...,...,...,...,...
439,2022,4,45,BTC,19784.976698
440,2022,4,46,BTC,19461.589161
441,2022,4,47,BTC,18986.625817
442,2022,4,48,BTC,18672.994275


In [11]:
pricer_class.db.connect()
relevant_columns = list(set(list(training_data.columns)) - set(pricer_class.factors))
pricer_class.db.store("sim",training_data[relevant_columns])
pricer_class.db.disconnect()